In [9]:
# Setting up modules
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import geopy

In [10]:
gardens = pd.read_csv('../data/NYRP_gardens.csv', index_col=0)
gardens.head()

,address,coords,lat,long,borough
0,735 East 211th St,"40.877499,-73.863489",40.877499,-73.863489,Bronx
1,1818 Bathgate Ave,"40.845051,-73.897747",40.845051,-73.897747,Bronx
2,1017 Teller Ave,"40.82815,-73.914356",40.828150,-73.914356,Bronx
3,1328 Clay Ave,"40.834934,-73.908892",40.834934,-73.908892,Bronx
4,2044 Prospect Ave,"40.845818,-73.887851",40.845818,-73.887851,Bronx


In [11]:
gardens.dtypes # check that lat, long are numeric

address     object
coords      object
lat        float64
long       float64
borough     object
dtype: object

In [12]:
# create geometry column with shapely.geometry Point objects
gardens['geometry'] = gardens.apply(lambda gardens: Point((gardens.long, gardens.lat)), axis=1)

# create geodataframe for spatial join later
gardens_gdf = gpd.GeoDataFrame(gardens, geometry='geometry')
gardens_gdf.head()

,address,coords,lat,long,borough,geometry
0,735 East 211th St,"40.877499,-73.863489",40.877499,-73.863489,Bronx,POINT (-73.863489 40.877499)
1,1818 Bathgate Ave,"40.845051,-73.897747",40.845051,-73.897747,Bronx,POINT (-73.897747 40.845051)
2,1017 Teller Ave,"40.82815,-73.914356",40.828150,-73.914356,Bronx,POINT (-73.91435600000001 40.82815)
3,1328 Clay Ave,"40.834934,-73.908892",40.834934,-73.908892,Bronx,POINT (-73.90889200000001 40.834934)
4,2044 Prospect Ave,"40.845818,-73.887851",40.845818,-73.887851,Bronx,POINT (-73.887851 40.845818)


In [13]:
# nta data to join on
nta = gpd.read_file('../data/Neighborhood Tabulation Areas.geojson')
nta.head()

,ntacode,shape_area,county_fips,ntaname,shape_leng,boro_name,boro_code,geometry
0,BK88,54005018.7472,047,Borough Park,39247.2280737,Brooklyn,3,(POLYGON ((-73.97604935657381 40.6312759056467...
1,QN51,52488276.477,081,Murray Hill,33266.904811,Queens,4,(POLYGON ((-73.80379022888246 40.7756101117924...
2,QN27,19726845.26,081,East Elmhurst,19816.7115378,Queens,4,(POLYGON ((-73.86109724401859 40.7636644770877...
3,QN07,22887772.7683,081,Hollis,20976.3358374,Queens,4,(POLYGON ((-73.75725671509139 40.7181386016625...
4,MN06,10647077.6122,061,Manhattanville,17040.6865482,Manhattan,1,(POLYGON ((-73.94607828608069 40.8212632160616...


In [14]:
# check what projection nta dataset is using
nta.crs

{'init': 'epsg:4326'}

In [15]:
# set projection for gardens to a compatible one
gardens_gdf.crs = nta.crs

In [16]:
# spatial join
gardens_joined = gpd.sjoin(gardens_gdf, nta, op='intersects')
gardens_joined.head()

,address,coords,lat,long,borough,geometry,index_right,ntacode,shape_area,county_fips,ntaname,shape_leng,boro_name,boro_code
0,735 East 211th St,"40.877499,-73.863489",40.877499,-73.863489,Bronx,POINT (-73.863489 40.877499),75,BX44,36273595.3423,005,Williamsbridge-Olinville,27351.0773787,Bronx,2
1,1818 Bathgate Ave,"40.845051,-73.897747",40.845051,-73.897747,Bronx,POINT (-73.897747 40.845051),128,BX01,16451615.9732,005,Claremont-Bathgate,29972.7777202,Bronx,2
2,1017 Teller Ave,"40.82815,-73.914356",40.828150,-73.914356,Bronx,POINT (-73.91435600000001 40.82815),175,BX14,18222401.7791,005,East Concourse-Concourse Village,27223.8477217,Bronx,2
3,1328 Clay Ave,"40.834934,-73.908892",40.834934,-73.908892,Bronx,POINT (-73.90889200000001 40.834934),175,BX14,18222401.7791,005,East Concourse-Concourse Village,27223.8477217,Bronx,2
4,2044 Prospect Ave,"40.845818,-73.887851",40.845818,-73.887851,Bronx,POINT (-73.887851 40.845818),144,BX17,19255162.9005,005,East Tremont,29633.136729,Bronx,2


In [17]:
# sanity check for spatial join
f"number of gardens with different borough specified={len(gardens_joined[~(gardens_joined.borough == gardens_joined.boro_name)])}"

'number of gardens with different borough specified=0'

In [19]:
# great! now saving the csv
gardens_joined.to_csv('../data/NYRP_NTA.csv')